In [ ]:
# ============================================
# CODICE 3 -  Merge POC + SuperTrend multiperiodo
# ============================================
 
import subprocess
import sys
 
# Funzione per installare i pacchetti
def install_packages():
    packages = ['yfinance', 'pandas', 'numpy', 'TA-Lib']
    for package in packages:
        try:
            # Suppress output for pip install
            with open(os.devnull, 'w') as f:
                subprocess.check_call([sys.executable, '-m', 'pip', 'install', package], stdout=f, stderr=f)
            print(f"Installato {package} con successo")
        except:
            print(f"Errore nell'installazione di {package}")
 
# Installa i pacchetti necessari
install_packages()
 
# Importa le librerie
try:
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import talib as ta
    from datetime import datetime, timedelta
    from dateutil.relativedelta import relativedelta
    from google.colab import drive
    import os
    import contextlib
except ImportError as e:
    print(f"Errore nell'importazione delle librerie: {e}")
    # Non uscire qui, permetti all'utente di vedere l'errore e risolverlo
    # sys.exit(1)
 
 
# === 1. Monta Drive e carica file ===
# Suppress output for drive mount
with open(os.devnull, 'w') as f:
  with contextlib.redirect_stdout(f), contextlib.redirect_stderr(f):
    # Aggiunto controllo per evitare di montare se già montato
    if not os.path.exists('/content/drive'):
      drive.mount('/content/drive')
    else:
      print("Google Drive già montato.")
 
 
week_number = datetime.now().isocalendar()[1]
# Assicurati che 'poc_period' sia definito e corretto, usa un valore predefinito se necessario
# Potresti voler recuperare 'poc_period' dalla cella precedente se non è globale
try:
  # Tenta di recuperare poc_period dalla cella precedente o usa un default
  if 'poc_period' not in globals():
      poc_period = '5y' # Default value if not found
 
  poc_folder = '/content/drive/MyDrive/POC'
  # Assicurati che il percorso del file corrisponda esattamente a quello di salvataggio
  poc_file_path = os.path.join(poc_folder, f'POC{poc_period}_week_{week_number}.xlsx')
 
  # Suppress output when loading the Excel file
  with open(os.devnull, 'w') as f:
    with contextlib.redirect_stdout(f), contextlib.redirect_stderr(f):
      df_poc = pd.read_excel(poc_file_path)
 
  print("File POC caricato con successo.")
 
except FileNotFoundError:
    print(f"Errore: File POC non trovato in {poc_file_path}.")
    print("Assicurati che la cella precedente sia stata eseguita correttamente e che il file esista.")
    # Non uscire qui, permetti al codice di continuare (anche se con un dataframe vuoto)
    df_poc = pd.DataFrame() # Crea un dataframe vuoto per evitare errori successivi
except Exception as e:
    print(f"Errore caricamento file POC: {e}")
    # Non uscire qui
    df_poc = pd.DataFrame() # Crea un dataframe vuoto per evitare errori successivi
 
 
# Procedi solo se il dataframe POC non è vuoto
if not df_poc.empty:
 
  # === 2. Funzioni di supporto per SuperTrend TradingView ===
  def calculate_supertrend(high, low, close, atr_period, multiplier):
      """Calcola il Supertrend"""
 
      # Ensure inputs are numpy arrays
      high = np.asarray(high)
      low = np.asarray(low)
      close = np.asarray(close)
 
      if len(high) < atr_period or len(low) < atr_period or len(close) < atr_period or atr_period <= 0:
          # print("Dati insufficienti per calcolare Supertrend.") # Opzionale: stampa un messaggio
          return np.full_like(close, np.nan), np.full_like(close, np.nan)
 
      # Calcolo ATR
      atr = ta.ATR(high, low, close, timeperiod=atr_period)
 
      # Calcolo delle bande
      upperband = (high + low) / 2 + (multiplier * atr)
      lowerband = (high + low) / 2 - (multiplier * atr)
 
      # Inizializza Supertrend
      supertrend = np.full_like(close, np.nan)
      direction = np.full_like(close, np.nan)
 
      # Find the first valid index after ATR calculation
      first_valid_atr_idx = np.where(~np.isnan(atr))[0][0] if np.any(~np.isnan(atr)) else len(close)
 
      if first_valid_atr_idx >= len(close):
          # print("ATR non calcolabile per periodo specificato.") # Opzionale: stampa un messaggio
          return np.full_like(close, np.nan), np.full_like(close, np.nan)
 
      # Initialize first valid Supertrend value
      supertrend[first_valid_atr_idx] = upperband[first_valid_atr_idx]
      direction[first_valid_atr_idx] = 1
 
 
      # Calcola Supertrend
      for i in range(first_valid_atr_idx + 1, len(close)):
          # Use previous valid supertrend value if the current one is nan (can happen with short data)
          prev_supertrend = supertrend[i-1] if not np.isnan(supertrend[i-1]) else (upperband[i-1] + lowerband[i-1]) / 2 # Fallback if prev is nan
 
          if close[i] > prev_supertrend:
              direction[i] = 1
          else:
              direction[i] = -1
 
          if direction[i] == 1:
              if lowerband[i] > prev_supertrend:
                  supertrend[i] = lowerband[i]
              else:
                  supertrend[i] = prev_supertrend
          else:
              if upperband[i] < prev_supertrend:
                  supertrend[i] = upperband[i]
              else:
                  supertrend[i] = prev_supertrend
 
      # Backfill NaNs at the beginning with the first valid Supertrend value
      first_valid_st_idx = np.where(~np.isnan(supertrend))[0][0] if np.any(~np.isnan(supertrend)) else len(close)
      if first_valid_st_idx < len(close):
          supertrend[:first_valid_st_idx] = supertrend[first_valid_st_idx]
 
 
      return supertrend, direction
 
  def calculate_start_date(interval):
      """Calcola la data di inizio per il download dei dati in base all'intervallo."""
      today = datetime.now()
      if interval == "1d":
          # 6 mesi per il daily
          return today - relativedelta(months=6)
      elif interval == "1wk":
          # 1 anno per il weekly
          return today - relativedelta(years=1)
      elif interval == "1mo":
          # 2 anni per il monthly
          return today - relativedelta(years=2)
      elif interval == "4h":
          # 60 giorni per il 4h
          return today - timedelta(days=60)
      else:
          return None # O gestisci altri intervalli se necessario
 
  def fix_df(df):
      """Rimuove MultiIndex e converte colonne in float"""
      if isinstance(df.columns, pd.MultiIndex):
          df.columns = [col[0] for col in df.columns]
      for col in ['Open','High','Low','Close','Adj Close','Volume']:
          if col in df.columns:
              # Use errors='coerce' to turn problematic values into NaN
              df[col] = pd.to_numeric(df[col], errors='coerce')
              # Drop rows with NaN in critical columns if necessary, or handle NaNs later
              # df.dropna(subset=[col], inplace=True) # Uncomment if dropping rows with NaN is desired
 
      # Ensure index is datetime
      if not isinstance(df.index, pd.DatetimeIndex):
          df.index = pd.to_datetime(df.index, errors='coerce')
          df.dropna(inplace=True) # Drop rows where index couldn't be converted
 
      return df
 
 
  # === 3. Funzione che calcola ST per 4H/Daily/Weekly/Monthly ===
  def get_supertrend_signals(ticker, atr_period=10, multiplier=3.0):
      try:
          # Suppress output when downloading data
          with open(os.devnull, 'w') as f:
            with contextlib.redirect_stdout(f), contextlib.redirect_stderr(f):
              start_date_daily = calculate_start_date("1d")
              start_date_weekly = calculate_start_date("1wk")
              start_date_monthly = calculate_start_date("1mo")
              start_date_4h = calculate_start_date("4h")
 
              # Added check for start_date being None
              df_daily = fix_df(yf.download(ticker, start=start_date_daily, interval="1d", auto_adjust=True) if start_date_daily else pd.DataFrame())
              df_weekly = fix_df(yf.download(ticker, start=start_date_weekly, interval="1wk", auto_adjust=True) if start_date_weekly else pd.DataFrame())
              df_monthly = fix_df(yf.download(ticker, start=start_date_monthly, interval="1mo", auto_adjust=True) if start_date_monthly else pd.DataFrame())
              df_4h = fix_df(yf.download(ticker, start=start_date_4h, interval="4h", auto_adjust=True) if start_date_4h else pd.DataFrame())
 
 
          # Added checks for required columns after fixing df
          if df_daily.empty or 'Close' not in df_daily.columns or 'High' not in df_daily.columns or 'Low' not in df_daily.columns:
              print(f"Dati giornalieri insufficienti o incompleti per {ticker}")
              st_daily_signal = "Err"
              close_price = np.nan # Use NaN if daily data is missing
          else:
              st_daily, dir_daily = calculate_supertrend(df_daily['High'], df_daily['Low'], df_daily['Close'], atr_period, multiplier)
              # Check if the last Supertrend value is not NaN before checking the signal
              st_daily_last = st_daily[-1] if not np.isnan(st_daily[-1]) else np.nan
              close_price = df_daily['Close'].iloc[-1]
              st_daily_signal = "V" if not np.isnan(st_daily_last) and close_price > st_daily_last else "R" if not np.isnan(st_daily_last) else "Err"
 
 
          if df_weekly.empty or 'Close' not in df_weekly.columns or 'High' not in df_weekly.columns or 'Low' not in df_weekly.columns:
              print(f"Dati settimanali insufficienti o incompleti per {ticker}")
              st_weekly_signal = "Err"
          else:
              st_weekly, dir_weekly = calculate_supertrend(df_weekly['High'], df_weekly['Low'], df_weekly['Close'], atr_period, multiplier)
              st_weekly_last = st_weekly[-1] if not np.isnan(st_weekly[-1]) else np.nan
              st_weekly_signal = "V" if not np.isnan(st_weekly_last) and close_price > st_weekly_last else "R" if not np.isnan(st_weekly_last) else "Err"
 
 
          if df_monthly.empty or 'Close' not in df_monthly.columns or 'High' not in df_monthly.columns or 'Low' not in df_monthly.columns:
              print(f"Dati mensili insufficienti o incompleti per {ticker}")
              st_monthly_signal = "Err"
          else:
              st_monthly, dir_monthly = calculate_supertrend(df_monthly['High'], df_monthly['Low'], df_monthly['Close'], atr_period, multiplier)
              st_monthly_last = st_monthly[-1] if not np.isnan(st_monthly[-1]) else np.nan
              st_monthly_signal = "V" if not np.isnan(st_monthly_last) and close_price > st_monthly_last else "R" if not np.isnan(st_monthly_last) else "Err"
 
 
          if df_4h.empty or 'Close' not in df_4h.columns or 'High' not in df_4h.columns or 'Low' not in df_4h.columns:
              print(f"Dati 4h insufficienti o incompleti per {ticker}")
              st_4h_signal = "Err"
          else:
              st_4h, dir_4h = calculate_supertrend(df_4h['High'], df_4h['Low'], df_4h['Close'], atr_period, multiplier)
              st_4h_last = st_4h[-1] if not np.isnan(st_4h[-1]) else np.nan
              st_4h_signal = "V" if not np.isnan(st_4h_last) and close_price > st_4h_last else "R" if not np.isnan(st_4h_last) else "Err"
 
 
          return st_4h_signal, st_daily_signal, st_weekly_signal, st_monthly_signal
 
      except Exception as e:
          print(f"Errore durante il calcolo del SuperTrend per {ticker}: {e}")
          return "Err","Err","Err","Err"
 
  # === 4. Applica ai ticker del file POC ===
  df_poc[['ST_4H','ST_Daily','ST_Weekly','ST_Monthly']] = df_poc['Ticker'].apply(
      lambda x: pd.Series(get_supertrend_signals(x))
  )
 
  print("\nTabella finale con segnali SuperTrend:")
  # Controlla se ci sono colonne SuperTrend prima di stampare
  supertrend_cols = ['ST_4H','ST_Daily','ST_Weekly','ST_Monthly']
  if any(col in df_poc.columns for col in supertrend_cols):
      print(df_poc.round(1).to_string())
  else:
      print("Nessun dato SuperTrend calcolato.")
 
 
  # === 5. Esporta in Excel ===
  output_dir = '/content/drive/MyDrive/Supertrend'
  os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist
 
  output_file_path = os.path.join(output_dir, f'SuperTrend_POC{poc_period}_week_{week_number}.xlsx')
  df_poc.to_excel(output_file_path, index=False)
 
  print(f"\nTabella esportata in: {output_file_path}")
 
else:
  print("Skipping SuperTrend calculation as POC file was not loaded.")
  # Puoi aggiungere qui un'alternativa o semplicemente terminare lo script in modo pulito
 